# Demo de Simulación de Balance - Cantidad de Inversiones

## Configuración inicial de librerías

In [1]:
# Contiene las clases Operacion, Caja y Banco.
import simula_balance as sb

# Contiene las reglas de renovación, por ejemplo,
# toda la caja a overnight.
import reglas_renovacion as rn

# Contiene las métricas, por ejemplo, el C46.
import metricas as met

# Se usa pandas sólo para desplegar claramente el gap de liquidez.
import pandas as pd

## t = 0: Configuración inicial del Banco

In [17]:
# Monto de capital
capital = 10.0

# Sobre este objeto caen los vencimientos de activos y pasivos.
# De aquí se obtiene el dinero para captaciones y colocaciones,
# toda operación, al iniciarse, tiene su correspondiente movimiento
# en la caja.
caja = sb.Caja()

# Construimos un Banco muy sencillo con una colocación bullet
# y una captación (también bullet).
# Lo primero son las tasas de mercado
mercado = {'on_col': 0.0, 'on_cap':0.0,
           'depo2':0.0312, 'depo3':0.033, 'depo4':0.0348,
           'inversion':0.027, 'colocacion':0.045}

# Variable que establece los plazos y porcentajes de renovación a cada plazo.
# Si tenemos x:y significa que al plazo x (medido en número de períodos)
# se hace un y% del monto necesario.
plazos = {4:100}


# Se construye el crédito
credito = sb.Operacion(mercado['colocacion'], 0, 12, 100, {'id':'1', 'ap':'activo', 'producto':'credito'})

# En esta lista se almacenarán las operaciones que insertaremos inicialmente en el banco.
operaciones = [credito]

# Se construye la mezcla de depósitos
i = 0
for plazo in plazos:
    i += 1
    codigo_tasa = 'depo' + str(plazo)
    ponderador = plazos[plazo] / 100.0
    operaciones.append(sb.Operacion(mercado[codigo_tasa], 0, plazo, ponderador * 90,
                                    {'id':str(i), 'ap':'pasivo', 'producto':'depo'}))
    
# Se da de alta el banco inicial
banco = sb.Banco(caja, operaciones, capital)

### Se construye y muestra el gap de liquidez inicial del banco.

In [18]:
# Se determina el gap de liquidez del Banco.
gap = met.construye_gap_liquidez(banco)

# El gap de liquidez se guarda en un dataframe para imprimirlo bien.
pd_gap = pd.DataFrame([gap()])

# Se muestra el resultado
print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
print pd_gap
print 'c46: ', met.c46(banco, True)

Gap de Liquidez a tiempo t =  0
    0   1   2   3       4   5   6   7   8   9   10  11     12  13  14  15  16
0  0.0   0   0   0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0
c46:  -0.0


### Se muestra el estado de resultados inicial.

In [19]:
# Se calcula y muestra, no se almacena.
banco.estado_resultados()

{'activo': {'intereses_devengados': 0.0, 'intereses_recibidos': 0.0},
 'pasivo': {'intereses_devengados': 0.0, 'intereses_pagados': 0.0}}

### Se muestra el balance inicial.

In [20]:
# Se calcula y muestra, no se almacena.
banco.balance()

{'activo': {'caja': 0.0, 'credito': 100.0},
 'pasivo': {'depo': -90.0, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

Vamos a definir algunas funciones auxiliares que harán menos verboso el ejercicio:

In [21]:
def muestra_gap():
    # se determina la situación de liquidez y
    gap = met.construye_gap_liquidez(banco)
    pd_gap = pd.DataFrame([gap()])

    # se muestra el resultado.
    print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
    print pd_gap
    print 'c46: ', met.c46(banco, True)
    return

def muestra_resultados():
    # Calcula y muestra el estado de resultados
    er = banco.estado_resultados()
    resultado = 0.0
    for key in er:
        print key, er[key]
        if key == 'activo':
            for interes in er[key]:
                resultado += er[key][interes]
        else:
            for interes in er[key]:
                resultado -= er[key][interes]
    print "resultado: ", resultado
    return

def muestra_balance():
    # Calcula y muestra el balance
    bal = banco.balance()
    for key in bal:
        print key, bal[key]
    return

def muestra_operaciones(t):
    # Muestra los flujos de las operaciones del banco
    activos = banco.activos()
    print 'Activos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for act in activos:
        print act.atributos()['producto'], act.vencimiento(), act.vencimiento() - t, act.monto(), act.interes_devengado(t), act.flujo()
        
    pasivos = banco.pasivos()
    print 'Pasivos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for pas in pasivos:
        print pas.atributos()['producto'], pas.vencimiento(), pas.vencimiento() - t, pas.monto(), pas.interes_devengado(t), pas.flujo()

def avanza():
    # Se avanza un período en el tiempo,
    banco.avanza()

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    
    print
    
    # Se calcula y muestra el EERR.
    muestra_resultados()
    
    print
    
    # Se calcula y muestra el balance
    muestra_balance()
        
    print
    
    # Se muestran las operaciones
    muestra_operaciones(banco.tiempo_actual())
    return
    
def renueva_depo():
    # Se renueva el depósito
    rn.renueva_depos(banco, plazos)

    # se determina la nueva situación de liquidez y
    # se muestra
    muestra_gap()
    return

def toma_inversiones():
    # Se renueva la inversión
    rn.inversion(banco)

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    return

def arregla_c46():
    # Se arregla el C46,
    rn.cumple_c46(banco, plazos, True)

    # Se muestra gap
    muestra_gap()

In [22]:
muestra_operaciones(0)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 12.0 100.0 0.0 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 4.0 90.0 0.0 93.132


## t = 1: Se avanza un período en el tiempo y se muestra la nueva situación de liquidez, el EERR, el balance y las operaciones (vigentes y vencidas).

In [23]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  1
    0   1   2       3   4   5   6   7   8   9   10     11  12  13  14  15  16
0  0.0   0   0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0   0
c46:  -0.0

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.783}
resultado:  0.342

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 101.125}
pasivo {'depo': -90.783, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 11.0 100.0 1.125 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 3.0 90.0 0.783 93.132


### Se "arregla" el C46. Se compran inversiones financiándolas con depósitos a los plazos indicados en la variable **plazos**.

In [24]:
muestra_resultados()

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.783}
resultado:  0.342


## t = 2: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [25]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  2
    0   1       2   3   4   5   6   7   8   9      10  11  12  13  14  15  16
0  0.0   0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0   0   0
c46:  -0.0

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 1.566}
resultado:  0.684

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 102.25}
pasivo {'depo': -91.566, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 10.0 100.0 2.25 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 2.0 90.0 1.566 93.132


In [11]:
# Notar que el nuevo depo es por un monto menor a la inversión porque partimos con caja > 0.
muestra_operaciones(2)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 10.0 100.0 2.25 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 2.0 90.0 1.566 93.132


In [12]:
muestra_resultados()

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 1.566}
resultado:  0.684


## t = 3: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [26]:
# Se avanza un período en el tiempo
avanza()

Gap de Liquidez a tiempo t =  3
    0       1   2   3   4   5   6   7   8      9   10  11  12  13  14  15  16
0  0.0 -93.132   0   0   0   0   0   0   0  113.5   0   0   0   0   0   0   0
c46:  93.132

activo {'intereses_devengados': 3.375, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 2.349}
resultado:  1.026

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 103.375}
pasivo {'depo': -92.349, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 9.0 100.0 3.375 113.5
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 1.0 90.0 2.349 93.132


### Se arregla el C46.

In [27]:
# Se arregla el C46,
arregla_c46()

Gap de Liquidez a tiempo t =  3
    0       1   2   3       4   5   6   7   8      9   10  11  12  13  14  15  \
0  0.0  0.6286   0   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6286


In [28]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 9.0 100.0 3.375 113.5
inversion 4.0 1.0 93.132 0.0 93.7606
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 1.0 90.0 2.349 93.132
depo 7.0 4.0 93.132 0.0 96.373


In [29]:
muestra_resultados()

activo {'intereses_devengados': 3.375, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 2.349}
resultado:  1.026


## t = 4. Volvemos a avanzar.

In [30]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  4
       0   1   2       3   4   5   6   7      8   9   10  11  12  13  14  15  \
0  0.6286   0   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6286

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 0.6286}
pasivo {'intereses_pagados': 3.132, 'intereses_devengados': 0.8102}
resultado:  1.1864

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.6286, 'credito': 104.5}
pasivo {'depo': -93.9422, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 8.0 100.0 4.5 113.5
inversion 4.0 0.0 93.132 0.0 93.7606
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 0.0 90.0 0.0 93.132
depo 7.0 3.0 93.132 0.8102 96.373


In [31]:
muestra_resultados()

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 0.6286}
pasivo {'intereses_pagados': 3.132, 'intereses_devengados': 0.8102}
resultado:  1.1864


## t = 5: Volvemos a avanzar.

In [32]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  5
       0   1       2   3   4   5   6      7   8   9   10  11  12  13  14  15  \
0  0.6286   0 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.6286

activo {'intereses_devengados': 5.625, 'intereses_recibidos': 0.6286}
pasivo {'intereses_pagados': 3.132, 'intereses_devengados': 1.6205}
resultado:  1.5011

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.6286, 'credito': 105.625}
pasivo {'depo': -94.7525, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 7.0 100.0 5.625 113.5
inversion 4.0 -1.0 93.132 0.0 93.7606
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 -1.0 90.0 0.0 93.132
depo 7.0 2.0 93.132 1.6205 96.373


## t = 6: Volvemos a avanzar.

In [33]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  6
       0       1   2   3   4   5      6   7   8   9   10  11  12  13  14  15  \
0  0.6286 -96.373   0   0   0   0  113.5   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  95.7444

activo {'intereses_devengados': 6.75, 'intereses_recibidos': 0.6286}
pasivo {'intereses_pagados': 3.132, 'intereses_devengados': 2.4307}
resultado:  1.8159

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.6286, 'credito': 106.75}
pasivo {'depo': -95.5627, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 6.0 100.0 6.75 113.5
inversion 4.0 -2.0 93.132 0.0 93.7606
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 -2.0 90.0 0.0 93.132
depo 7.0 1.0 93.132 2.4307 96.373


In [34]:
arregla_c46()

Gap de Liquidez a tiempo t =  6
    0       1   2   3        4   5      6   7   8   9   10  11  12  13  14  \
0 -0.0  0.0177   0   0 -98.4258   0  113.5   0   0   0   0   0   0   0   0   

   15  16  
0   0   0  
c46:  -0.0177


## t = 7: Volvemos a avanzar

In [35]:
avanza()

Gap de Liquidez a tiempo t =  7
       0   1   2        3   4      5   6   7   8   9   10  11  12  13  14  15  \
0  0.0177   0   0 -98.4258   0  113.5   0   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.0177

activo {'intereses_devengados': 7.875, 'intereses_recibidos': 1.2749000000000001}
pasivo {'intereses_pagados': 6.373, 'intereses_devengados': 0.8275}
resultado:  1.9494

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0177, 'credito': 107.875}
pasivo {'depo': -95.9433, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 5.0 100.0 7.875 113.5
inversion 4.0 -3.0 93.132 0.0 93.7606
inversion 7.0 0.0 95.7444 0.0 96.3907
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 -3.0 90.0 0.0 93.132
depo 7.0 0.0 93.132 0.0 96.373
depo 10.0 3.0 95.1158 0.8275 98.4258


## t = 8. Última iteración.

In [36]:
avanza()

Gap de Liquidez a tiempo t =  8
       0   1        2   3      4   5   6   7   8   9   10  11  12  13  14  15  \
0  0.0177   0 -98.4258   0  113.5   0   0   0   0   0   0   0   0   0   0   0   

   16  
0   0  
c46:  -0.0177

activo {'intereses_devengados': 9.0, 'intereses_recibidos': 1.2749000000000001}
pasivo {'intereses_pagados': 6.373, 'intereses_devengados': 1.655}
resultado:  2.2469

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0177, 'credito': 109.0}
pasivo {'depo': -96.7708, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 4.0 100.0 9.0 113.5
inversion 4.0 -4.0 93.132 0.0 93.7606
inversion 7.0 -1.0 95.7444 0.0 96.3907
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 -4.0 90.0 0.0 93.132
depo 7.0 -1.0 93.132 0.0 96.373
depo 10.0 2.0 95.1158 1.655 98.4258


In [38]:
muestra_operaciones(8)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 12.0 4.0 100.0 9.0 113.5
inversion 4.0 -4.0 93.132 0.0 93.7606
inversion 7.0 -1.0 95.7444 0.0 96.3907
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 4.0 -4.0 90.0 0.0 93.132
depo 7.0 -1.0 93.132 0.0 96.373
depo 10.0 2.0 95.1158 1.655 98.4258


In [39]:
muestra_resultados()

activo {'intereses_devengados': 9.0, 'intereses_recibidos': 1.2749000000000001}
pasivo {'intereses_pagados': 6.373, 'intereses_devengados': 1.655}
resultado:  2.2469
